<a href="https://colab.research.google.com/github/bruninho-design/AjudaOSul.AI-ImersaoAlura/blob/main/AjudaOSulAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q -U google-generativeai
!python3 -m -q pip install --upgrade pip
!python3 -m -q pip install --upgrade Pillow

from google.colab import userdata
import pathlib
import textwrap
import google.generativeai as genai

# Configure sua API Key
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

  system_instruction = "Olá! Sou AjudaOSul.AI, um sistema inteligente dedicado a combater fraudes em pedidos de ajuda durante situações de crise. Minha missão é analisar descrições e imagens, avaliando sua autenticidade para garantir que a ajuda chegue a quem realmente precisa."

# Modelo Generativo para análise de texto e imagem
model = genai.GenerativeModel(model_name="gemini-pro")

# Função para analisar pedidos de ajuda e gerar relatório com score
def analisar_pedido(descricao, url_imagem=None):
  # Prompt para a IA
  prompt = f"""Você é AjudaOSul.AI, um assistente de investigação especializado em identificar fraudes em pedidos de ajuda durante crises.
  Receberá descrições e imagens e deve analisá-las para determinar se são autênticas ou fraudulentas.
  Para cada entrada, forneça uma análise detalhada dos elementos textuais e visuais que embasam sua conclusão.

  Exemplo de Entrada para a IA:
  Descrição: Urgente! Nossa casa foi destruída pelas enchentes, precisamos de alimentos e abrigo imediatamente.
  Imagem URL: [URL mostrando casa destruída] (opcional)
  Pergunta: O pedido menciona crianças?

  Ações Esperadas da IA:
  * **Análise Textual:** Verificar urgência e desespero na descrição.
  Comparar padrões linguísticos com fraudes conhecidas.
  Identificar sentimentos e elementos específicos de emergência.
  * **Análise Visual:**
  Identificar sinais de edição ou manipulação.
  Verificar se a imagem é consistente com a descrição.
  Analisar metadados para confirmar data e local, se disponível.
  * **Análise do Conteúdo:** Verificar se crianças são mencionadas.

  Critérios de Avaliação (Score):
  * **Contexto da Imagem (40 pontos):** Imagem mostra de forma clara os efeitos da enchente, alinhando-se à descrição.
  * **Descrição do Pedido (30 pontos):** Descrição contém palavras-chave como 'enchente' e 'desabrigado'.
  * **Presença de Crianças (15 pontos):** Pedido menciona crianças afetadas.
  * **Alinhamento entre Imagem e Descrição (10 pontos):** Imagem e descrição corroboram a situação relatada.
  * **Palavras-Chave (5 pontos):** Identificação de palavras-chave específicas relacionadas à catástrofe.

  Exemplo de Saída:
  * **Contexto da Imagem:** 40/40 pontos. Imagem mostra claramente os efeitos da enchente.
  * **Descrição do Pedido:** 25/30 pontos. Descrição menciona a necessidade de abrigo e comida.
  * **Presença de Crianças:** 15/15 pontos. Menciona três crianças assustadas.
  * **Alinhamento entre Imagem e Descrição:** 10/10 pontos. Imagem e texto são coerentes.
  * **Palavras-Chave:** 5/5 pontos. Uso adequado de palavras como 'enchente' e 'abrigar'.

  Score: 95/100
  * **Análise: "A imagem e a descrição corroboram os efeitos da enchente, mencionando crianças e fornecendo uma descrição detalhada, sugerindo um pedido verdadeiro."
  * **Conclusão: Verdadeiro

  Recomendações Finais:
  * **Verifique a Legitimidade:** Doe para organizações confiáveis.
  * **Plataformas Seguras:** Utilize canais seguros para doar.
  * **Eficiência nas Doações:** Prefira doações financeiras ou de suprimentos específicos.
  * **Acompanhe o Uso:** Monitore a transparência da organização.
  * **Compartilhe Informação:** Divulgue a iniciativa para alcançar mais doadores.

  **Verificar pedido de ajuda: ** {descricao}
  """
  if url_imagem:
      prompt += f"\n**Verificar pedido de ajuda: ** (imagem em anexo: {url_imagem})"

  prompt += "\n\n**Chatbot:**"

  # Enviar prompt para o modelo e obter resposta
  response = model.generate_content(prompt)

  # Exibir relatório da IA em formato de chat
  print(response.text)

# Loop do chat
while True:
  # Obter entrada do usuário
  descricao = input("Verificar pedido de ajuda: ")
  url_imagem = input("Verificar pedido de ajuda: (URL da imagem - opcional): ")
  criancas = input("O pedido menciona crianças? (responda 'Sim' ou 'Não'): ")

  # Verificar se o usuário deseja sair
  if descricao.lower() == "sair":
    break

  # Analisar o pedido e gerar relatório em formato de chat
  analisar_pedido(descricao, url_imagem)

  print("-" * 50)

/usr/bin/python3: No module named -q
/usr/bin/python3: No module named -q
Verificar pedido de ajuda: Nossa casa está inundada há dias. Precisamos de comida e abrigo imediatamente. Tenho três crianças comigo que estão muito assustadas.
Verificar pedido de ajuda: (URL da imagem - opcional): https://s2-g1.glbimg.com/etcEBaVu7SKgJ0l00yAPbTrE5LE=/0x0:5000x3333/1000x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2024/A/p/PV6edmSYea9wDnLQVgSw/2024-05-09t045400z-255888473-rc2jm7a5av6t-rtrmadp-3-brazil-rains.jpg
O pedido menciona crianças? (responda 'Sim' ou 'Não'): sim
**Contexto da Imagem:** 40/40 pontos. A imagem mostra claramente os efeitos das inundações, com água cobrindo uma área residencial e danos visíveis às casas.

**Descrição do Pedido:** 30/30 pontos. A descrição é curta, mas menciona a necessidade de comida e abrigo imediatamente, transmitindo uma sensação de urgência e desespero.

**Presença de Crianças:** 15/15 pontos. O p